In [16]:
!pip install dask
!pip install CANdas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import candas as cd
import pandas as pd
from functools import reduce
import time
import dask

In [18]:
start_time = time.time()
'''!unzip /content/data/4x12_CESL_Proto1_Delhi_Cycle_Trails.zip -d /content/data'''

'!unzip /content/data/4x12_CESL_Proto1_Delhi_Cycle_Trails.zip -d /content/data'

In [19]:
db = cd.load_dbc("/content")
# This is the BLF file 'my_file.blf':
log = cd.from_file(dbc_db=db,filename= '/content/D1F95')

Loading dbc...
Finished loading.
Converting data to readable format... this might take several minutes
The following IDs caused errors: {420511872, 420577408, 420512129, 420577665, 420642944, 420643201, 1793, 129, 417998720, 417998721, 1298, 419361047, 419415321, 218043431, 217056039, 217056295, 217055783, 419366695, 419366439, 419366183, 218043175, 419360295, 418184999, 418119463, 419377907, 419427060, 218057203, 218057715, 218058227, 218058739, 218079475, 218080243, 218080755, 419407091, 536870911}


In [20]:
all_keys=log.keys()

In [ ]:
all_keys=[ i for i in all_keys]
all_keys

In [22]:
battery_keys=[i for i in all_keys if ('attery' in i) | ('ell' in i)]
battery_keys

['HV_Battery_Temp_1',
 'HV_Battery_Temp_2',
 'HV_Battery_Temp_3',
 'HV_Battery_Temp_4',
 'maxBatteryVoltage',
 'minBatteryVoltage',
 'A_Max_Cell_Volt',
 'A_Min_Cell_Volt',
 'A_Max_Cell_Temp',
 'A_Min_Cell_Temp',
 'A_Max_Cell_Volt_No',
 'A_Min_Cell_Volt_No',
 'B_Max_Cell_Volt',
 'B_Min_Cell_Volt',
 'B_Max_Cell_Temp',
 'B_Min_Cell_Temp',
 'B_Max_Cell_Volt_No',
 'B_Min_Cell_Volt_No',
 'D_Max_Cell_Volt',
 'D_Min_Cell_Volt',
 'D_Max_Cell_Temp',
 'D_Min_Cell_Temp',
 'D_Max_Cell_Volt_No',
 'D_Min_Cell_Volt_No',
 'A_Cell_OverVolt_Alarm',
 'A_Cell_UnderVolt_Alarm',
 'A_Cell_OverTemp_Alarm',
 'A_Cell_UnderTemp_Alarm',
 'A_Cell_volt_sampling_Alarm',
 'A_Cell_temp_sampling_Alarm',
 'A_Cell_DV_Static_Alarm',
 'A_Cell_DV_Dyn_Alarm',
 'A_Cell_Delta_Temperture_Alarm',
 'A_Cell_Balancing_Hw_Flt',
 'A_Max_Cell_Temp_No',
 'A_Min_Cell_Temp_No',
 'B_Cell_OverVolt_Alarm',
 'B_Cell_UnderVolt_Alarm',
 'B_Cell_OverTemp_Alarm',
 'B_Cell_UnderTemp_Alarm',
 'B_Cell_volt_sampling_Alarm',
 'B_Cell_temp_sampling_Ala

In [23]:
req_keys=['A_Pack_Curr_Value','B_Pack_Curr_Value','C_Pack_Curr_Value','D_Pack_Curr_Value','A_Pack_Voltage_Value',
 'B_Pack_Voltage_Value','C_Pack_Voltage_Value','D_Pack_Voltage_Value','B_Mux_Voltage_Counter','B_Cell_Voltage_N',
 'B_Cell_Voltage_N_plus_1','B_Cell_Voltage_N_plus_2','B_Cell_Voltage_N_plus_3','B_Cell_Voltage_N_plus_4']
req_dict={}

In [24]:
import dask.dataframe as dd
from datetime import datetime, timedelta

In [25]:
def date_con(x):
  return pd.to_datetime(x, unit='s')

In [26]:
all_dict={}
for i in all_keys[4:]:
  try:
    all_dict[i]=pd.DataFrame(data=log[i])
    all_dict[i].columns=['Time',i]
    all_dict[i]=dd.from_pandas(all_dict[i],npartitions=1000)
    all_dict[i].Time=all_dict[i].Time.map_partitions(date_con)
  except:
    all_dict.pop(i)

In [27]:
all_dict['A_Pack_Curr_Value'].head()

,Time,A_Pack_Curr_Value
0,2023-02-24 08:56:01.279971584,-2.0
1,2023-02-24 08:56:01.329776640,-1.0
2,2023-02-24 08:56:01.379700992,0.0
3,2023-02-24 08:56:01.429766400,2.0
4,2023-02-24 08:56:01.479615488,3.0


In [28]:
final_csv=reduce(lambda x,y: dd.merge(x,y, on='Time', how='outer'), [all_dict[i] for i in all_dict.keys()])

In [ ]:
final_csv.to_parquet(
    '/content/data',
     write_index=False,
    engine="pyarrow",
    write_metadata_file=False,)

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))